# **Adhesion Flow Simulator**

---

## Overview

The **Adhesion Flow Simulator** is a computational tool designed to simulate and visualize the behavior of cells under the influence of fluid flow. This pipeline models cell attachment to endothelial surfaces, guided by factors like flow speed, adhesion strength, and receptor map data. The simulator integrates biophysical properties of cells and fluid dynamics, offering insights into cell behavior in various conditions.

The simulations use real-time flow adjustments and cell attachment dynamics to generate data, which can be visualized through heatmaps, flow fields, and Ripley’s L function analysis.

## Features

- **Flow-Driven Simulations**: Simulate cell movement and attachment in a fluid environment with tunable flow speeds.
- **Receptor Map Support**: Load receptor map images to introduce spatial heterogeneity in cell adhesion probabilities.
- **Ripley’s L Function**: Analyze spatial point patterns using Ripley’s L function to assess clustering of attached cells.
- **Heatmaps and Visualization**: Generate visual outputs like flow fields, heatmaps, and cell trajectory videos.
- **Checkpointing**: Save and resume simulations from checkpoints to handle large-scale runs efficiently.


--------------------------------------------------------
# **Part 0. Prepare the Google Colab session**
--------------------------------------------------------
<font size = 4>skip this section when using a local installation


## **0.1. Install key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to install


print("In progress....")

%pip install --quiet phiflow

#!git clone https://github.com/CellMigrationLab/AdhesionFlowSimulator.git



In [ ]:
!unzip /content/AdhesionFlowSimulator-main.zip -d /content/


## **0.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the instructions.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab
from google.colab import drive
drive.mount('/content/Gdrive')
# This command was originally but I think it doesn't do anything really
## %cd /gdrive

--------------------------------------------------------
# **Part 1. Prepare the session**
--------------------------------------------------------

In [ ]:
# @title ##Run to load key functions

import os
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itertools import product
import time
import pandas as pd
import cv2
from phi.jax.flow import *
import json
import sys
import requests
from scipy.spatial import distance_matrix

sys.path.append("../")
sys.path.append("AdhesionFlowSimulator/")

import AdhesionFlowSimulator
from AdhesionFlowSimulator import *
from src.flow_utils import *
from src.io_utils import *
from src.simulations import *
from src.cell_dynamics import *
from src.spatial_analysis import *

# Current version of the notebook the user is running
current_version = "0.1"
Notebook_name = 'AdhesionFlowSimulator'

# URL to the raw content of the version file in the repository
version_url = "https://raw.githubusercontent.com/guijacquemet/AdhesionFlowSimulator/main/notebooks/latest_version.txt"

# Function to define colors for formatting messages
class bcolors:
    WARNING = '\033[91m'  # Red color for warning messages
    ENDC = '\033[0m'      # Reset color to default

# Check if this is the latest version of the notebook
try:
    All_notebook_versions = pd.read_csv(version_url, dtype=str)
    print('Notebook version: ' + current_version)

    # Check if 'Version' column exists in the DataFrame
    if 'Version' in All_notebook_versions.columns:
        Latest_Notebook_version = All_notebook_versions[All_notebook_versions["Notebook"] == Notebook_name]['Version'].iloc[0]
        print('Latest notebook version: ' + Latest_Notebook_version)

        if current_version == Latest_Notebook_version:
            print("This notebook is up-to-date.")
        else:
            print(bcolors.WARNING + "A new version of this notebook has been released. We recommend that you download it at https://github.com/guijacquemet/CellTracksColab" + bcolors.ENDC)
    else:
        print("The 'Version' column is not present in the version file.")
except requests.exceptions.RequestException as e:
    print("Unable to fetch the latest version information. Please check your internet connection.")
except Exception as e:
    print("An error occurred:", str(e))



# **Part 2. Choose the simulation settings**


# Simulation Configuration Documentation

This document provides an overview of the key parameters used in the simulation setup, describing their purpose and how they influence the behavior of the simulation.

## Space Configuration

- **PIXEL_SIZE**: `1.31`  
  Defines the size of each pixel in microns. This is used to scale the spatial measurements of the simulation to real-world units.

- **CELL_AREA_MIN**: `240`  
  Minimum area of a cell in square microns. Cells in the simulation will have an area between this value and `CELL_AREA_MAX`.

- **CELL_AREA_MAX**: `350`  
  Maximum area of a cell in square microns. Together with `CELL_AREA_MIN`, this defines the range for cell sizes.

- **FRAME_INTERVAL**: `0.04`  
  The time between each frame in seconds. This determines the temporal resolution of the simulation.

- **field_size**: `(512, 512)`  
  The size of the simulation field (or viewport) in pixels. This determines the area in which the simulation occurs.

- **grid_size**: Same as `field_size`  
  Represents the resolution of the grid used for spatial calculations. In this case, it's the same as the field size.

## Simulation Configuration

- **NUM_FRAMES**: `500`  
  The total number of frames the simulation will run for. This defines the duration of the simulation in discrete time steps.

- **time_steps**: `NUM_FRAMES`  
  Alias for `NUM_FRAMES`. Represents the number of time steps the simulation will execute.

- **FLOW_SPEEDS**: `[100]`  
  A list of flow speeds (in microns per second) to be used in the simulation. Different flow speeds will simulate different environmental conditions.

- **adhesion_strengths**: `[0, 0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2]`  
  A list of values representing the adhesion strength of cells. Higher values indicate stronger adhesion to the surface.

- **cell_densities**: `[0.0003]`  
  The density of cells in the simulation, expressed as cells per square micron. This controls how many cells are placed in the field.

- **num_runs**: `1`  
  The number of simulation runs to perform for each parameter combination.

## Attachment Background

- **background_value**: `[0.3]`  
  Defines a uniform adhesion threshold used when a receptor map is not loaded. Cells must exceed this threshold to attach.

- **use_receptor_map**: `False`  
  Set to `True` if you want to load a receptor map image to control the spatial variation in adhesion properties. If `False`, a uniform background will be used.

- **receptor_map_image_folder**:  
  Specifies the directory path where the receptor map images are stored, if `use_receptor_map` is set to `True`.

## Flow Recalculation After Attachment

- **disable_flow_recompute**: `True`  
  If set to `False`, the flow field is recalculated after each cell attachment to account for changes caused by arrested cells. Set to `True` to disable this recalculation and save computation time.

## Output Configuration

- **result_folder**:  
  The directory where the simulation results (e.g., cell positions, attachment counts, flow patterns) will be saved. This ensures all output files are organized and accessible.

- **create_video**: `False`  
  If set to `True`, the simulation will generate videos of the cell dynamics. This can provide a visual representation of the simulation.

## Debug Configuration

- **debug_mode**: `False`  
  When set to `True`, the simulation will display each created frame for visualization purposes, which can help with debugging and understanding the simulation's progress.


In [ ]:
# Space Configuration
PIXEL_SIZE = 1.31  # microns per pixel
CELL_AREA_MIN = 240  # micron^2
CELL_AREA_MAX = 350  # micron^2
FRAME_INTERVAL = 0.04  # seconds
field_size = (512, 512)  # Size of the field of view in pixels
grid_size = field_size

# Simulation Configuration

NUM_FRAMES = 500 # Number of frame the simulation will run for
time_steps = NUM_FRAMES  # Number of time steps for the simulation
FLOW_SPEEDS = [100]  # Flow speeds in microns per second
adhesion_strengths = [0, 0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2]
cell_densities = [0.0003]
num_runs = 1

# Attachement background
background_value = [0.3] # Adhesion threshold used when not loading a receptor map
use_receptor_map = False  # Set to True if using a receptor map image
receptor_map_image_folder = '/content/Gdrive/Shareddrives/PDAC extravasation project 3/Simulations/CD44_FOV/512'

#Compute the local flow after attachement
disable_flow_recompute=True # Set to False to compute the flow field after each arrest

# Outputs
result_folder = '/content/test2'
os.makedirs(result_folder, exist_ok=True)

create_video = False  # Set to True if you want to generate videos

# Debug
debug_mode=False # Set to True to see each created frame

print(f"""
Simulation Settings Loaded:

Space Configuration:
  - Pixel Size: {PIXEL_SIZE} microns
  - Cell Area: {CELL_AREA_MIN}-{CELL_AREA_MAX} micron^2
  - Field Size: {field_size} pixels
  - Frame Interval: {FRAME_INTERVAL} seconds

Simulation Configuration:
  - Number of Frames: {NUM_FRAMES}
  - Flow Speeds: {FLOW_SPEEDS} microns/second
  - Adhesion Strengths: {adhesion_strengths}
  - Cell Densities: {cell_densities} cells/micron^2
  - Number of Runs: {num_runs}

Attachment Background:
  - Using Receptor Map: {use_receptor_map}
  - Background Value: {background_value} (if receptor map is disabled)

Flow Recalculation After Attachment:
  - Disable Flow Recompute: {disable_flow_recompute}

Outputs:
  - Results will be saved in: {result_folder}
  - Video Creation: {create_video}

Debug:
  - Debug Mode: {debug_mode}
""")


In [ ]:
# @title ##Run to prepare your simulations


# Cell size calculations
cell_diameter_min = int(np.sqrt((CELL_AREA_MIN / np.pi)) * 2 / PIXEL_SIZE)  # pixels
cell_diameter_max = int(np.sqrt((CELL_AREA_MAX / np.pi)) * 2 / PIXEL_SIZE)  # pixels
cell_diameter_avg = int((cell_diameter_min + cell_diameter_max) / 2)  # pixels

# Calculate the average cell area in pixels^2
cell_area_avg = int(((CELL_AREA_MIN + CELL_AREA_MAX) / 2) / (PIXEL_SIZE ** 2))  # pixels^2

# Calculate the cell radius in pixels
cell_radius = cell_diameter_avg // 2  # pixels

positions_folder = os.path.join(result_folder, 'Positions')
counts_folder = os.path.join(result_folder, 'Counts')
attachment_matrix_folder = os.path.join(result_folder, 'Attachment_Matrices')
flow_pattern_folder = os.path.join(result_folder, 'Flow_Pattern')
attached_cells_folder = os.path.join(result_folder, 'attached_cells')

for folder in [positions_folder, counts_folder, attachment_matrix_folder, flow_pattern_folder, attached_cells_folder]:
    os.makedirs(folder, exist_ok=True)

# Load or Create Checkpoint
config_file_path = os.path.join(result_folder, 'simulation_checkpoint.json')
if os.path.exists(config_file_path):
    with open(config_file_path, 'r') as config_file:
        completed_simulations = json.load(config_file)
    print(f"Found {len(completed_simulations)} completed simulations.")
else:
    completed_simulations = []
    print("No completed simulations found. Starting fresh.")


# Generate Simulation Key
def generate_simulation_key(flow_speed, adhesion_strength, cell_density, run_id, mask_name):
    return f'{flow_speed}_{adhesion_strength}_{cell_density}_{run_id}_{mask_name}'

# Generate the list of masks or use "uniform"
def get_masks_or_uniform():
    if use_receptor_map:
        return [f for f in os.listdir(receptor_map_image_folder) if f.endswith('.tif')]
    else:
        return ['uniform']  # Use "uniform" as a placeholder

# Prepare all simulation parameters with the mask name included
def create_simulation_keys():
    mask_list = get_masks_or_uniform()
    simulation_params = list(product(FLOW_SPEEDS, adhesion_strengths, cell_densities, range(num_runs), mask_list))
    all_simulation_keys = [generate_simulation_key(*params) for params in simulation_params]
    return all_simulation_keys, mask_list

# Simulation setup
all_simulation_keys, mask_list = create_simulation_keys()

# Print information about completed and remaining simulations
remaining_simulations = [key for key in all_simulation_keys if key not in completed_simulations]
print(f"Total simulations: {len(all_simulation_keys)}")
print(f"Remaining simulations: {len(remaining_simulations)}")

# **Part 3. Run your simulations**


In [ ]:
# @title ##Run to start your simulations


for key in tqdm(remaining_simulations, desc="Simulations"):
    # Extract parameters from the key
    flow_speed, adhesion_strength, cell_density, run_id, mask_name = key.split('_')

    # Load the mask or create uniform background
    if mask_name == "uniform":
        background = np.ones(field_size) * background_value  # Uniform background
        print(f"Using uniform background with value: {background_value}")
    else:
        receptor_map_image_path = os.path.join(receptor_map_image_folder, mask_name)
        background = load_receptor_map(receptor_map_image_path, field_size)
        print(f"Using receptor map: {mask_name}")

    # Run the simulation
    flow_rate_per_frame = calculate_flow_rate_per_frame(float(flow_speed), FRAME_INTERVAL, PIXEL_SIZE)
    space = Space(field_size=field_size, grid_size=grid_size, initial_flow_speed=flow_rate_per_frame)
    total_attached_cells_count = run_simulation(
        result_folder,
        field_size,
        grid_size,
        PIXEL_SIZE,
        time_steps,
        cell_diameter_avg,
        cell_radius,
        space,
        flow_rate_per_frame=flow_rate_per_frame,
        flow_speed=float(flow_speed),
        adhesion_strength=float(adhesion_strength),
        cell_density=float(cell_density),
        positions_folder=positions_folder,
        counts_folder=counts_folder,
        attachment_matrix_folder=attachment_matrix_folder,
        flow_pattern_folder=flow_pattern_folder,
        attached_cells_folder=attached_cells_folder,
        run_id=int(run_id),
        background=background,
        create_video=create_video,
        debug_mode=debug_mode,
        disable_flow_recompute=disable_flow_recompute,
        mask_name=mask_name )

    # Save the completed simulation
    completed_simulations.append(key)
    with open(config_file_path, 'w') as config_file:
        json.dump(completed_simulations, config_file)

print("All simulations completed")


# **Part 4. Count the number of attached cells**


In [ ]:
result_folder = '/content/test2' # @param {type: "string"}


In [ ]:

# @title ##Count the number of attached cells per conditions

# Usage
count_folder = os.path.join(result_folder, 'Counts')

# Call the function to sum attached cells over time and save to CSV
sum_attached_cells_over_time(result_folder, count_folder)


In [ ]:

# @title ##Plots the number of attached cells per conditions

plot_heatmaps_total_attached(result_folder)


# **Part 5. Spatial analyses of attached cells**


### Spatial Analysis Settings

- **`result_folder`**:
  - Path where all simulation results, outputs, and figures are saved. This directory will contain various output files including the heatmaps and statistical data from the spatial analysis.
  - **Example**: `'/content/test2'`.

- **`field_size`**:
  - Represents the size of the simulation or observation area in pixels. This determines the spatial bounds within which the analysis is conducted.
  - **Example**: `(512, 512)` (Width and height in pixels).

- **`FRAME_INTERVAL`**:
  - The time interval between two consecutive frames of the simulation. This value is essential for time-based calculations in the analysis, such as dynamic behaviors of cells over time.
  - **Example**: `0.04` seconds (i.e., 40 milliseconds between frames).

- **`PIXEL_SIZE`**:
  - Defines the physical size of a pixel in microns. This is used to convert between pixel measurements and real-world distances. This is crucial for spatial analyses where measurements need to be made in physical units.
  - **Example**: `1.31` microns per pixel.

- **`radii`**:
  - A list of radii values in micrometers used for spatial analysis such as Ripley's K or L-function. These radii determine the distances at which spatial relationships are examined. Ripley's functions calculate how the spatial distribution of points (cells) changes across these distances.
  - **Example**: `np.arange(1, 200, 10)` (This generates a range of radii from 1 to 200 microns, spaced by 10 microns).



In [ ]:
result_folder = '/content/test2'
field_size = (512, 512)  # Size of the field of view in pixels
FRAME_INTERVAL = 0.04  # seconds
PIXEL_SIZE = 1.31  # microns per pixel
radii = np.arange(1, 200, 10)  # Example radii in micrometers


In [ ]:

# @title ##Compute the Ripley's values for each simulation

# Main script to compute Ripley's L function
compute_ripley_l(result_folder, radii, field_size, FRAME_INTERVAL, PIXEL_SIZE)


In [ ]:
# @title ##Plots the Ripley_L_Values per conditions

radius = 51 # @param {type: "integer"}


# Replace result_folder with the path to your result folder
plot_heatmaps_ripley_l(result_folder, radius=radius)


# **Part 6. Utils functions and playground**


## Test the flow computation with obstacle

In [ ]:
from phi.jax.flow import *
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from tqdm import tqdm

# @title ###Test


# Define stationary cell obstacles
cells = [
    Sphere(x=50, y=25, radius=5),  # Cell 1
    Sphere(x=70, y=30, radius=5),  # Cell 2
]

# Plotting the obstacles (optional)
plot({"Cells": union(*cells)['x,y']})
plt.show()

# Set water-like properties at 37°C
viscosity = 0.0007  # In Pa·s
density = 993  # Density in kg/m³ for water at 37°C

pbar = tqdm(total=200)

@jit_compile
def step(v, p, dt=1.):
    v = advect.semi_lagrangian(v, v, dt)
    v, p = fluid.make_incompressible(v, cells, Solve(x0=p))
    pbar.update(1)
    return v, p

# Define 2D boundary conditions for unilateral, infinite flow
boundary = {
    'x-': vec(x=8, y=0),  # Strong inflow from the left
    'x+': ZERO_GRADIENT,  # Outflow on the right (zero gradient for smooth exit)
    'y-': ZERO_GRADIENT,  # Open boundary at the bottom
    'y+': ZERO_GRADIENT   # Open boundary at the top
}

# Initialize the velocity grid in 2D with uniform flow to simulate an infinite source
v0 = StaggeredGrid((8., 0), boundary, x=128, y=64, bounds=Box(x=200, y=100))

# Run the simulation
v_trj, p_trj = iterate(step, batch(time=200), v0, None)

# Visualize the flow field in 2D (curl of the velocity field)
v_trj_2d = v_trj[{'vector': 'x,y'}]
anim = plot(v_trj_2d.time[100:].curl(), animate='time')
anim.save('cells_curl.gif')
plt.show()

# Plot the final velocity field with cells overlaid
plot(v_trj_2d.time[-1], union(*cells)['x,y'], overlay='args')
plt.show()


In [ ]:
from phi.jax.flow import *
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from tqdm import tqdm

# @title ###Test


# Define stationary cell obstacles
cells = [
    Sphere(x=50, y=25, radius=5),  # Cell 1
    Sphere(x=70, y=30, radius=5),  # Cell 2
]

# Plotting the obstacles (optional)
plot({"Cells": union(*cells)['x,y']})
plt.show()

# Set water-like properties at 37°C
viscosity = 0.0007  # In Pa·s
density = 993  # Density in kg/m³ for water at 37°C

pbar = tqdm(total=200)

@jit_compile
def step(v, p, dt=1.):
    v = advect.semi_lagrangian(v, v, dt)
    v = diffuse.explicit(v, viscosity, dt)

    v, p = fluid.make_incompressible(v, cells, Solve(x0=p))
    pbar.update(1)
    return v, p

# Define 2D boundary conditions for unilateral, infinite flow
boundary = {
    'x-': vec(x=8, y=0),  # Strong inflow from the left
    'x+': ZERO_GRADIENT,  # Outflow on the right (zero gradient for smooth exit)
    'y-': ZERO_GRADIENT,  # Open boundary at the bottom
    'y+': ZERO_GRADIENT   # Open boundary at the top
}

# Initialize the velocity grid in 2D with uniform flow to simulate an infinite source
v0 = StaggeredGrid((8., 0), boundary, x=128, y=64, bounds=Box(x=200, y=100))

# Run the simulation
v_trj, p_trj = iterate(step, batch(time=200), v0, None)

# Visualize the flow field in 2D (curl of the velocity field)
v_trj_2d = v_trj[{'vector': 'x,y'}]
anim = plot(v_trj_2d.time[100:].curl(), animate='time')
anim.save('cells_curl.gif')
plt.show()

# Plot the final velocity field with cells overlaid
plot(v_trj_2d.time[-1], union(*cells)['x,y'], overlay='args')
plt.show()


## Plot the Plots the Ripley_L_Values per conditions (code for cell line data)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# @title ###Plots the Ripley_L_Values per conditions (code for cell line data)


# Function to plot Ripley's L Function heatmap by Cells and Flow Speed
def plot_heatmap_ripley_l(csv_file_path):
    # Load the CSV file with Ripley's L values
    ripley_l_df = pd.read_csv(csv_file_path)

    # Ensure 'Flow_speed' and 'Cells' are treated as categorical variables
    ripley_l_df['Flow_speed'] = ripley_l_df['Flow_speed'].astype(str)
    ripley_l_df['Cells'] = ripley_l_df['Cells'].astype(str)

    # Group the data by Flow Speed and Cells, and calculate the mean Ripley's L value
    grouped_df = ripley_l_df.groupby(
        ['Flow_speed', 'Cells']
    )['Ripley_L_at_Specific_Radius_slow'].mean().reset_index()

    print("Grouped DataFrame head:\n", grouped_df.head())

    # Pivot the DataFrame for heatmap
    heatmap_data = grouped_df.pivot(
        index='Cells', columns='Flow_speed', values='Ripley_L_at_Specific_Radius_slow'
    )

    # Plot heatmap
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_facecolor('yellow')  # Set the background color of the axes

    # Plot the heatmap with chosen colormap, setting limits from -50 to +50
    cax = ax.imshow(heatmap_data, cmap='coolwarm', aspect='auto', vmin=-50, vmax=50)

    # Set titles and labels
    ax.set_title("Ripley's L Function Heatmap")
    ax.set_xlabel('Flow Speed')
    ax.set_ylabel('Cells')

    # Customize the ticks
    ax.set_xticks(np.arange(len(heatmap_data.columns)))
    ax.set_xticklabels(heatmap_data.columns)
    ax.set_yticks(np.arange(len(heatmap_data.index)))
    ax.set_yticklabels(heatmap_data.index)

    # Add colorbar with label
    cbar = fig.colorbar(cax, ax=ax, orientation='vertical')
    cbar.set_label("Ripley L Value")

    # Save the heatmap to a file
    plt.savefig(os.path.join(os.path.dirname(csv_file_path), 'heatmap_ripley_l_values.pdf'))
    plt.show()
    plt.close(fig)

    print(f"Heatmap plotting completed. Results saved to {os.path.dirname(csv_file_path)}")

# Replace 'csv_file_path' with the actual path to your CSV file in Google Colab
csv_file_path = '/content/ripleys_l_values.csv'
plot_heatmap_ripley_l(csv_file_path)


## Load the receptor masks and compute the averages

In [ ]:
# @title ###load the masks and compute the averages


receptor_map_image_folder = '/content/Gdrive/Shareddrives/PDAC extravasation project 3/Simulations/CD44_FOV/512'  # @param {type: "string"}

# Call the function to load the masks and compute the averages
overall_average = load_masks_and_compute_average(receptor_map_image_folder)
